
Predicting Fake News using NLP and Machine Learning 

Reperformance using Data set with fake news

https://towardsdatascience.com/predicting-fake-news-using-nlp-and-machine-learning-scikit-learn-glove-keras-lstm-7bbd557c3443


In [2]:
!pip install contractions

     |████████████████████████████████| 287 kB 27.7 MB/s 
     |████████████████████████████████| 106 kB 40.6 MB/s 


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['figure.figsize'] = [10,10]
import seaborn as sns
sns.set_theme(style="darkgrid")
from wordcloud import WordCloud

import nltk
from nltk import sent_tokenize
''' Required for Google Colab'''
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
''' Required for Google Colab'''
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize

import contractions
import re
import itertools
import datetime
import time
from collections import Counter
import string

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
train_df = pd.read_csv('https://raw.githubusercontent.com/smkerr/COVID-fake-news-detection/main/data/Constraint_Train.csv', header=0)
val_df = pd.read_csv('https://raw.githubusercontent.com/smkerr/COVID-fake-news-detection/main/data/Constraint_Val.csv', header=0)

In [6]:
train_df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6420 non-null   int64 
 1   tweet   6420 non-null   object
 2   label   6420 non-null   object
dtypes: int64(1), object(2)
memory usage: 150.6+ KB


In [8]:
def preprocess_text(x):
  cleaned_text = re.sub(r'[^a-zA-Z\d\s\']+', '', x)
  word_list = []
  for each_word in cleaned_text.split(' '):
    try:
      word_list.append(contractions.fix(each_word).lower())
    except:
      print(x)
  return " ".join(word_list)

In [9]:
text_cols = ['tweet']

In [10]:
%%time
for col in text_cols:
  print("Processing column: {}".format(col))
  train_df[col] = train_df[col].apply(lambda x: preprocess_text(x))
  val_df[col] = val_df[col].apply(lambda x: preprocess_text(x)) 

Processing column: tweet
CPU times: user 1.5 s, sys: 11.2 ms, total: 1.51 s
Wall time: 1.7 s


In [11]:
%%time
for col in text_cols:
  print("Processing column: {}".format(col))
  train_df[col] = train_df[col].apply(word_tokenize)
  val_df[col] = val_df[col].apply(word_tokenize)

Processing column: tweet
CPU times: user 1.63 s, sys: 22.1 ms, total: 1.65 s
Wall time: 1.67 s


In [12]:
%%time
for col in text_cols:
  print("Processing column: {}".format(col))
  train_df[col] = train_df[col].apply(lambda x: [each_word for each_word in x if each_word not in stopwords])
  val_df[col] = val_df[col].apply(lambda x: [each_word for each_word in x if each_word not in stopwords])

Processing column: tweet
CPU times: user 767 ms, sys: 713 µs, total: 768 ms
Wall time: 973 ms


In [13]:
train_df.head() 

,id,tweet,label
0,1,"[cdc, currently, reports, 99031, deaths, gener...",real
1,2,"[states, reported, 1121, deaths, small, rise, ...",real
2,3,"[politically, correct, woman, almost, uses, pa...",fake
3,4,"[indiafightscorona, 1524, covid, testing, labo...",real
4,5,"[populous, states, generate, large, case, coun...",real


## POS Tag Counts

In [14]:
all_tokenized_gen = [a for b in train_df[train_df['label']=="real"]['tweet'].tolist() for a in b]
all_tokenized_fake = [a for b in train_df[train_df['label']=="fake"]['tweet'].tolist() for a in b]

In [15]:
def get_post_tags_list(tokenized_articles):
  all_pos_tags = []
  for word in tokenized_articles:
    pos_tag = nltk.pos_tag([word])[0][1]
    all_pos_tags.append(pos_tag)
  return all_pos_tags

In [16]:
%%time
all_pos_tagged_word_gen = get_post_tags_list(all_tokenized_gen)
all_pos_tagged_word_fake = get_post_tags_list(all_tokenized_fake)

CPU times: user 15.9 s, sys: 907 ms, total: 16.8 s
Wall time: 17.2 s


In [17]:
all_pos_tagged_word_gen[:5], all_pos_tagged_word_fake[:5]

(['NN', 'RB', 'NNS', 'CD', 'NNS'], ['RB', 'NN', 'NN', 'RB', 'NNS'])

In [18]:
gen_pos_df = pd.DataFrame(dict(Counter(all_pos_tagged_word_gen)).items(), columns=['Pos_tag', 'Genuine News'])
fake_pos_df = pd.DataFrame(dict(Counter(all_pos_tagged_word_fake)).items(), columns=['Pos_tag', 'Fake News'])

In [19]:
gen_pos_df

,Pos_tag,Genuine News
0,NN,32854
1,RB,1937
2,NNS,11298
3,CD,6586
4,JJ,5703
5,VBD,849
6,VBG,2269
7,VBN,3424
8,IN,787
9,JJR,168


In [20]:
fake_pos_df

,Pos_tag,Fake News
0,RB,1155
1,NN,24778
2,NNS,5887
3,JJ,3007
4,VB,582
5,IN,376
6,VBN,1985
7,VBD,583
8,CD,1374
9,PRP,144


In [21]:
pos_df = gen_pos_df.merge(fake_pos_df, on='Pos_tag')

In [22]:
pos_df['Genuine News'] = pos_df['Genuine News'] * 100 / pos_df['Genuine News'].sum()
pos_df['Fake News'] = pos_df['Fake News'] * 100 / pos_df['Fake News'].sum()
pos_df.head()

,Pos_tag,Genuine News,Fake News
0,NN,47.937550,58.329998
1,RB,2.826293,2.718991
2,NNS,16.485008,13.858612
3,CD,9.609688,3.234539
4,JJ,8.321296,7.078792


## Tf-idf and Count Vectorizer

In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['figure.figsize'] = [10,10]
import seaborn as sns
sns.set_theme(style="darkgrid")

import nltk
from nltk import sent_tokenize
''' Required for Google Colab'''
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
''' Required for Google Colab'''
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import contractions

import string
import joblib
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [24]:
train_df['tweet_joined'] = train_df['tweet'].apply(lambda x: " ".join(x))
val_df['tweet_joined'] = val_df['tweet'].apply(lambda x: " ".join(x))

In [25]:
target = train_df['label'].values

In [26]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
tf_idf_transformer = TfidfTransformer(smooth_idf=False)

# fit and transform train data to count vectorizer
count_vectorizer.fit(train_df['tweet_joined'].values)
count_vect_train = count_vectorizer.transform(train_df['tweet_joined'].values)

# fit the counts vector to tfidf transformer
tf_idf_transformer.fit(count_vect_train)
tf_idf_train = tf_idf_transformer.transform(count_vect_train)

# Transform the test data as well
count_vect_test = count_vectorizer.transform(val_df['tweet_joined'].values)
tf_idf_test = tf_idf_transformer.transform(count_vect_test)

In [27]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(tf_idf_train, target, random_state=0)

In [28]:
df_perf_metrics = pd.DataFrame(columns=['Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision', 'Recall', 'f1_score'])

In [29]:
df_perf_metrics = pd.DataFrame(columns=[
    'Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision',
    'Recall', 'f1_score', 'Training Time (secs'
])

In [30]:
models_trained_list = []

In [31]:
def get_perf_metrics(model, i):
    # model name
    model_name = type(model).__name__
    # time keeping
    start_time = time.time()
    print("Training {} model...".format(model_name))
    # Fitting of model
    model.fit(X_train, y_train)
    print("Completed {} model training.".format(model_name))
    elapsed_time = time.time() - start_time
    # Time Elapsed
    print("Time elapsed: {:.2f} s.".format(elapsed_time))
    # Predictions
    y_pred = model.predict(X_test)
    # Add to ith row of dataframe - metrics
    df_perf_metrics.loc[i] = [
        model_name,
        model.score(X_train, y_train),
        model.score(X_test, y_test),
        precision_score(y_test, y_pred, pos_label="real"),
        recall_score(y_test, y_pred, pos_label="real"),
        f1_score(y_test, y_pred, pos_label="real"), "{:.2f}".format(elapsed_time)
    ]
    # keep a track of trained models
    models_trained_list.append(model)
    print("Completed {} model's performance assessment.".format(model_name))

In [32]:
models_list = [LogisticRegression(),
               MultinomialNB(),
               RandomForestClassifier(),
               DecisionTreeClassifier(),
               GradientBoostingClassifier(),
               AdaBoostClassifier()]

In [33]:
%%time
for n, model in enumerate(models_list):
    get_perf_metrics(model, n)

Training LogisticRegression model...
Completed LogisticRegression model training.
Time elapsed: 1.06 s.
Completed LogisticRegression model's performance assessment.
Training MultinomialNB model...
Completed MultinomialNB model training.
Time elapsed: 0.02 s.
Completed MultinomialNB model's performance assessment.
Training RandomForestClassifier model...
Completed RandomForestClassifier model training.
Time elapsed: 17.92 s.
Completed RandomForestClassifier model's performance assessment.
Training DecisionTreeClassifier model...
Completed DecisionTreeClassifier model training.
Time elapsed: 2.38 s.
Completed DecisionTreeClassifier model's performance assessment.
Training GradientBoostingClassifier model...
Completed GradientBoostingClassifier model training.
Time elapsed: 44.64 s.
Completed GradientBoostingClassifier model's performance assessment.
Training AdaBoostClassifier model...
Completed AdaBoostClassifier model training.
Time elapsed: 19.30 s.
Completed AdaBoostClassifier model'

In [34]:
df_perf_metrics

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.979855,0.895950,0.917293,0.878752,0.897609,1.06
1,MultinomialNB,0.990862,0.891589,0.859324,0.945978,0.900571,0.02
2,RandomForestClassifier,1.000000,0.895950,0.961219,0.833133,0.892605,17.92
3,DecisionTreeClassifier,1.000000,0.847352,0.846698,0.861945,0.854253,2.38
4,GradientBoostingClassifier,0.896781,0.854829,0.889610,0.822329,0.854648,44.64
5,AdaBoostClassifier,0.867913,0.839875,0.869231,0.813926,0.840670,19.30
